## SimGRASP (Cmax objective) - Create experiment files for Ying instances

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import os

%matplotlib inline

In [ ]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

### List Ying instance files in the instance folder

In [ ]:
rootfolder = os.getcwd()
searchpath = os.path.join(rootfolder, 'instances', 'robust', 'ying', 'tail_format', '*.txt')
display(searchpath)
file_list = glob.glob(searchpath, recursive=False)
file_list

### Function to export the experiment dataframe to text delimited file 

In [ ]:
def export_instance_dataframe(df, distribution, variance, nruns):
    outputfolder = os.path.join(rootfolder, 'experiments', 'simgrasp', distribution)
    if not os.path.exists(outputfolder):
        os.makedirs(outputfolder)
    print('Saving file on folder: ' + outputfolder)
    fname = os.path.join(outputfolder, 'tests2run_stoc_ying_{}_var{}_{}runs.txt'.format(distribution, variance, nruns))
    with open(fname, 'w') as file_buff:
        if distribution == 'triangular' or distribution == 'uniform':
            # uniform and triangular distributions do not need the beta1 parameter
            file_buff.write('# instance | maxTime(sec)(not used) | nIter(not used) | distribution | beta1 (not used) | Variance | seed\r\n')
        elif distribution == 'geometric':
            file_buff.write('#instance maxTime(sec) nIter distribution beta1 beta2 seed\r\n')
        df.to_csv(file_buff, sep='\t', header=False, index=False)
    print('Saved: ' + fname)

### Process all instance filenames and append all data to a single dataframe

Each Ying instance file will be executed 25 times by the SimGRASP algorithm : `nruns = 25`.

In [ ]:
%%time

# # instance | maxTime(sec)(not used) | nIter(not used) | distribution (not used)| beta1 (not used) | Variance | seed
nruns = 25
for distribution in ['triangular']:  # distribution used inside SimGRASP NEHT procedure : triangular
    for variance in [0.1, 0.2, 0.3, 0.4, 0.5]: # The SimGRASP algorithm will need variance value as the uncertainty parameter
        instance_list = []
        for filename in file_list:
            instance = filename[filename.rfind(os.path.sep)+1:filename.rfind('_')]
            n = int(instance[2:5])
            alpha = int(instance[5:7])
            sequence = int(instance[7:9])
            # Regardless of the alpha value, the Ying instances we are dealing with have the same nominal (expected) processing times
            if alpha == 50:  # We will only use the nominal processing times to feed the SimGRASP algorithm
                #print('Processing instance ', instance, 'n = {}, alpha = {}, seq = {}'.format(n, alpha, sequence))
                for r in range(1, nruns+1):
                    # We need a differnt random seed for each run
                    str_seed = str(int.from_bytes(os.urandom(4), 'big'))
                    seed = int(str_seed[:6])
                    # Parameters maxTime and nIter are never used (SimGRASP calculates maxTime based on timeFactor and n)
                    if distribution == 'geometric':
                        beta1 = 0.1  # this is the same beta1 parameter of geom distr, used in SimGRASP paper
                        instance_list.append([instance, 1, 100000, distribution[0], beta1, variance, seed])
                    else:  # uniform and triangular distributions do not need the beta1 parameter
                        instance_list.append([instance, 1, 100000, distribution[0], 1, variance, seed])
        df = pd.DataFrame(instance_list, columns=['instance', 'maxTime(sec)', 'nIter', 'distribution', 'beta1', 'beta2', 'seed'])
        export_instance_dataframe(df, distribution, variance, nruns)

In [ ]:
df

In [ ]:
df.info()